**Modulo 5 : Regularización y Dropout**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Parte 5.3: Regularización L1 y L2 para Disminuir _Overfitting_

La regularización L1 y L2 son dos técnicas de regularización comunes que pueden reducir los efectos del sobreajuste. Ambos algoritmos pueden funcionar con una función objetivo o como parte del algoritmo de retropropagación. En ambos casos el algoritmo de regularización se adjunta al algoritmo de entrenamiento añadiendo un objetivo adicional.

Ambos algoritmos funcionan agregando una penalización de peso al entrenamiento de la red neuronal. Esta penalización anima a la red neuronal a mantener los pesos en valores pequeños. Tanto L1 como L2 calculan esta penalización de manera diferente. Para los algoritmos basados ​​en descenso de gradientes, como la retropropagación, puede agregar este cálculo de penalización a los gradientes calculados. Para el entrenamiento basado en funciones objetivas, como el recocido simulado, la penalización se combina negativamente con la puntuación objetiva.

Tanto L1 como L2 funcionan de manera diferente en la forma en que penalizan el tamaño de un peso. L2 obligará a los pesos a adoptar un patrón similar a una distribución gaussiana; el L1 forzará los pesos en un patrón similar a una distribución de Laplace, como se muestra en la Figura 5.L1L2.

**Figure 5.L1L2: L1 vs L2**
![L1 vs L2](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_9_l1_l2.png "L1 vs L2")

Como puede ver, el algoritmo L1 es más tolerante con los pesos más allá de 0, mientras que el algoritmo L2 es menos tolerante. Destacaremos otras diferencias importantes entre L1 y L2 en las siguientes secciones. También debe tener en cuenta que tanto L1 como L2 cuentan sus penalizaciones basándose únicamente en los pesos; no cuentan las penalizaciones sobre los valores de sesgo.

In [1]:
import pandas as pd
from scipy.stats import zscore

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Tratar Missing values en 'income'
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Clasificación
products = dummies.columns
y = dummies.values

In [2]:
########################################
# Regresión con L1/L2 Utilizando Keras
########################################

import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers

# Cross-validate
kf = KFold(5, shuffle=True, random_state=42)
    
oos_y = []
oos_pred = []
fold = 0

for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    #kernel_regularizer=regularizers.l2(0.01),
    
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], 
            activation='relu',
             activity_regularizer=regularizers.l1(1e-4))) # Oculta 1
    model.add(Dense(25, activation='relu', 
                    activity_regularizer=regularizers.l1(1e-4))) # Oculta 2
    model.add(Dense(y.shape[1],activation='softmax')) # Salida
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),
              verbose=0,epochs=500)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    # probabilidades brutas a la clase elegida (probabilidad más alta)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)        

    # fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # accuracy
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


# Cree la lista de predicción oos y calcule el error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # accuracy

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)



Fold #1
Fold score (accuracy): 0.635
Fold #2
Fold score (accuracy): 0.705
Fold #3
Fold score (accuracy): 0.6725
Fold #4
Fold score (accuracy): 0.65
Fold #5
Fold score (accuracy): 0.64
Final score (accuracy): 0.6605
